# Modeling

## Load Packages

In [1]:
from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

ModuleNotFoundError: No module named 'ydata_profiling'

In [2]:
pip install ydata_profiling

ERROR: Could not find a version that satisfies the requirement ydata_profiling (from versions: none)
ERROR: No matching distribution found for ydata_profiling
Note: you may need to restart the kernel to use updated packages.


In [34]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier
# import xgboost as xgb
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings

In [4]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

In [5]:
import altair as alt
import random
import warnings

In [115]:
warnings.filterwarnings('ignore')

## Load Data

In [6]:
cdc_survey = pd.read_csv('../data/cdc_nhanes_survey_responses_clean.csv')

# filter to pregnant moms
cdc_survey_pmom = cdc_survey[cdc_survey['has_been_pregnant'] == 1]
cdc_survey_pmom

,Unnamed: 0,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ290,ALQ151,...,has_health_insurance,has_health_insurance_gap,has_smoked_tabacco_last_5days,is_male,is_usa_born,has_diabetes,has_overweight_diagnosis,has_tried_to_lose_weight_12mo,has_been_pregnant,monthly_poverty_index
5,5,109284.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,0,0.0,0.0,1.0,1.0,1.0,NaN
7,7,109290.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,...,1.0,0.0,0.0,0,1.0,1.0,0.0,1.0,1.0,4.80
8,8,109291.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0,1.0,0.0,1.0,0.0,1.0,NaN
11,11,109295.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,4.37
14,14,109300.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,...,1.0,0.0,0.0,0,0.0,0.0,0.0,1.0,1.0,2.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8950,8950,124802.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,...,1.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,NaN
8951,8951,124803.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0,0.0,1.0,0.0,0.0,1.0,1.18
8957,8957,124812.0,1.0,6.000000e+00,3.0,8.000000e+00,10.0,9.0,5.397605e-79,2.0,...,0.0,NaN,0.0,0,1.0,0.0,0.0,0.0,1.0,3.66
8958,8958,124813.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,...,1.0,1.0,0.0,0,0.0,0.0,0.0,1.0,1.0,NaN


## Training Step

### Features to Use

In [7]:
all_columns = [
    # Depression screener
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks',
    # Alcohol & smoking
    'has_smoked_tabacco_last_5days',
    'alcoholic_drinks_past_12mo',    
    # Diet & Nutrition
    'how_healthy_is_your_diet',    
    'count_lost_10plus_pounds',
    'has_tried_to_lose_weight_12mo',       
    # Physical health & Medical History
    'count_days_seen_doctor_12mo',
    'duration_last_healthcare_visit',        
    'count_days_moderate_recreational_activity',   
    'count_minutes_moderate_recreational_activity',
    'count_minutes_moderate_sedentary_activity',
    'general_health_condition',    
    'has_diabetes',
    'has_overweight_diagnosis',         
    # Demographic data
    'food_security_level_household',   
    'food_security_level_adult',    
    'monthly_poverty_index_category',
    'monthly_poverty_index',
    'count_hours_worked_last_week',
    'age_in_years',   
    'education_level',
    'is_usa_born',    
    'has_health_insurance',
    'has_health_insurance_gap'   
]
len(all_columns)

33

In [8]:
# find columns where everything is null
# need to delete those
#cdc_survey_pmom = cdc_survey_pmom[all_columns]
#columns = cdc_survey_pmom.columns[cdc_survey_pmom.notnull().any() == False]
#columns

### Create Test & Train Datasets

In [9]:
def get_model_data(original_df, columns, test_size_prop=0.2):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    # add target column (MDD)
    cols_to_use = columns.copy()
    cols_to_use.insert(0, 'MDD')
    
    df_to_use = original_df[cols_to_use]
    
    # Create test & train data
    x = df_to_use.iloc[:,1:].values
    y = df_to_use['MDD'].values
    
    # SimpleImputer() = fill in missing values
    # note imputer may drop columns if no values exist for it
    imputer = SimpleImputer(strategy='median')  
    x = imputer.fit_transform(x)

    # RobustScaler() = scale features to remove outliers
    trans = RobustScaler()
    x = trans.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(
        x, 
        y, 
        test_size=test_size_prop, 
        random_state=42
    ) 
    
    return x_train, x_test, y_train, y_test

## Test Step

In [38]:
def plot_confusion_matrix(y_test, pred_labels):
    """
    Function that displays a confusion matrix for provided true and predicted classes
    """
    #print(f'cover type 1 and type 2 total correct {np.sum(np.diag(metrics.confusion_matrix(y_test, pred_labels))[:2])}')

    cm = confusion_matrix(y_test, pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    fig, ax = plt.subplots(figsize=(5,5))
    disp = disp.plot(include_values=True, cmap='viridis', ax=ax, xticks_rotation='horizontal')    
    plt.grid(False)
    plt.show()
    return

def get_performance_df(label_actual, label_pred, index_label):
    """
    Function to calculate performance metrics for model.
    Includes precision, recal, F1, & support.
    """
    performance_report = classification_report(label_actual, label_pred, output_dict=True)
    performance_report = performance_report['macro avg']
    result_table = pd.DataFrame(performance_report, index = [index_label])
    result_table = result_table.reset_index().rename(columns = {'index':'model'})
    return result_table

def baseline_models(
    x_train, 
    y_train, 
    x_test, 
    y_test,
    do_smote=True,
    show_confusion_matrix=False,
    show_score_dataframe=False):
    """
    Function that trains and makes predictions using 5 of the classifiers went over during the class.
    Meant as a helper function for easier testing of different modeling pipelines.
    """

    #  do_smote
    if do_smote == True:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)

    # K-Nearest Neighbors
    knn = KNeighborsClassifier()
    knn.fit(x_train, y_train)
    pred_labels_knn  = knn.predict(x_test)
    score_knn = get_performance_df(y_test, pred_labels_knn,'Knn')
    
    # Logistic Regression
    lm = LogisticRegression()
    lm.fit(x_train, y_train)
    pred_labels_lr  = lm.predict(x_test)
    score_lr = get_performance_df(y_test, pred_labels_lr,'Logistic Regression')
        
    # Bernoulii Naive Bayes
    bnb = BernoulliNB()
    bnb.fit(x_train, y_train)
    pred_labels_bnb  = bnb.predict(x_test)
    score_bnb = get_performance_df(y_test, pred_labels_bnb,'Bernoulli Naive Bayes')    
        
    # Gaussian Naive Bayes
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    pred_labels_gnb  = gnb.predict(x_test)
    score_gnb = get_performance_df(y_test, pred_labels_gnb,'Gaussian Naive Bayes')    

    # Random Forest
    rf = RandomForestClassifier(random_state=0)
    rf.fit(x_train, y_train)
    pred_labels_rf  = rf.predict(x_test)
    predictions_posterior_rf = rf.predict_proba(x_test)
    score_rf = get_performance_df(y_test, pred_labels_rf,'Random Forest')   
    
    #Decision Tree
    dt = DecisionTreeClassifier()
    dt.fit(x_train, y_train)
    pred_labels_dt = dt.predict(x_test)
    score_dt = get_performance_df(y_test, pred_labels_dt,'Decision Tree')

    #Gradient Boosting Classifier
    gb = GradientBoostingClassifier()
    gb.fit(x_train, y_train)
    pred_labels_gb = gb.predict(x_test)
    score_gb = get_performance_df(y_test, pred_labels_gb,'Gradient Boosting Classifier')
    
#     #XGBoost
#     xgboost = xgb.XGBClassifier()
#     xgboost.fit(x_train, y_train)
#     pred_labels_xgboost = gbt.predict(x_test)
#     score_xgboost = get_performance_df(y_test, pred_labels_xgboost,'XGBoost')
    
    # make dataframe with scores
    scores = pd.concat([score_knn, score_lr, score_bnb, score_gnb, score_rf, score_dt, score_gb])
    scores = scores.sort_values(by = 'recall', ascending=False)
    
    if show_score_dataframe:
        display(scores.style.set_table_attributes('style="font-size: 17px"').hide_index())
    
    if show_confusion_matrix:
        print('\nK-Nearest Neighbors Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_knn)
        print('Logistic Regression Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_lr)
        print('Bernoulli Naive Bayes Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_bnb)
        print('Gaussian Naive Bayes Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_gnb)
        print('Random Forest Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_rf)
        print('Decision Tree Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_dt)
        print('Gradient Boosting Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_gbt)
#         print('XGBoost Confusion Matrix')
#         plot_confusion_matrix(y_test, pred_labels_xgboost)

    return scores

### Baseline models (just depression screener)

In [39]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = all_columns[0:10] # just depression screener
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2729, 10)
(683, 10)
(2729,)
(683,)


In [40]:
baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model[['model','precision','recall','f1-score']]

,model,precision,recall,f1-score
0,Bernoulli Naive Bayes,0.609595,0.672248,0.627428
0,Gaussian Naive Bayes,0.619179,0.657007,0.633433
0,Decision Tree,0.567399,0.542002,0.548767
0,Logistic Regression,0.653195,0.524981,0.525183
0,Knn,0.552888,0.521080,0.522166
0,Random Forest,0.552096,0.512648,0.506990
0,Gradient Boosting Classifier,0.522612,0.502593,0.485650


In [41]:
baseline_model_with_smote = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_with_smote[['model','precision','recall','f1-score']]

,model,precision,recall,f1-score
0,Logistic Regression,0.589635,0.687149,0.597762
0,Bernoulli Naive Bayes,0.587726,0.677093,0.597050
0,Gaussian Naive Bayes,0.600932,0.675809,0.617413
0,Knn,0.577054,0.630221,0.587912
0,Decision Tree,0.613434,0.590182,0.599570
0,Gradient Boosting Classifier,0.611144,0.567637,0.580668
0,Random Forest,0.533867,0.519614,0.521242


### Baseline models (all variables)

In [42]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = all_columns
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2729, 33)
(683, 33)
(2729,)
(683,)


In [43]:
baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model[['model','precision','recall','f1-score']]

/Users/leej136/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,model,precision,recall,f1-score
0,Bernoulli Naive Bayes,0.620125,0.688796,0.640166
0,Gaussian Naive Bayes,0.613735,0.664627,0.630484
0,Decision Tree,0.569376,0.573948,0.571506
0,Gradient Boosting Classifier,0.619722,0.523357,0.523049
0,Logistic Regression,0.588745,0.515895,0.510511
0,Knn,0.552096,0.512648,0.506990
0,Random Forest,0.450808,0.498377,0.473400


In [172]:
baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model[['model','precision','recall','f1-score']]

,model,precision,recall,f1-score
0,Logistic Regression,0.594492,0.700451,0.603120
0,Gaussian Naive Bayes,0.586315,0.687306,0.590851
0,Bernoulli Naive Bayes,0.598842,0.674186,0.614727
0,Knn,0.566361,0.657116,0.555784
0,Random Forest,0.641773,0.536659,0.544620


In [49]:
baseline_model

,model,precision,recall,f1-score,support
0,Logistic Regression,0.590391,0.687960,0.598980,683
0,Bernoulli Naive Bayes,0.587726,0.677093,0.597050,683
0,Gaussian Naive Bayes,0.600932,0.675809,0.617413,683
0,Knn,0.587408,0.638338,0.600637,683
0,Random Forest,0.510287,0.505500,0.502845,683


### Test Different Train/Test Proportion

Just using depression screener cols

In [191]:
# compare performance with various train splits from [20%, 80%]
test_size_props = np.arange(0.2,0.8,0.01)

In [192]:
### Without SMOTE

# create base table
model_perf = pd.DataFrame(columns = ['model','precision','recall','f1-score','support'])

# iterate across prop splits and save performance
for prop in test_size_props:

    x_train, x_test, y_train, y_test = get_model_data(
        original_df = cdc_survey_pmom,
        columns = all_columns[0:10],
        test_size_prop = prop
    )
    
    mod_data = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
    mod_data['test_size_prop'] = prop
    model_perf = model_perf.append(mod_data)
    
### With SMOTE

# create base table
model_perf_smote = pd.DataFrame(columns = ['model','precision','recall','f1-score','support'])

# iterate across prop splits and save performance
for prop in test_size_props:

    x_train, x_test, y_train, y_test = get_model_data(
        original_df = cdc_survey_pmom,
        columns = all_columns[0:10],
        test_size_prop = prop
    )
    
    mod_data = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
    mod_data['test_size_prop'] = prop
    model_perf_smote = model_perf_smote.append(mod_data)    

In [193]:
alt.Chart(model_perf).mark_line().encode(
    x='test_size_prop:Q',
    y='f1-score:Q',
    color='model:N',
)

alt.Chart(...)

In [70]:
alt.Chart(model_perf_smote).mark_line().encode(
    x='test_size_prop:Q',
    y='f1-score:Q',
    color='model:N',
)

alt.Chart(...)

### Test Different # of columns

#### Get column feature importances when including all

In [197]:
def get_random_forest_n_top_features(train_data=x_train, 
                                     train_labels=y_train):
    """
    A model that trains a random forest model and returns the feature importances as a dataframe.
    The index of the row corresponds to the column index in the initial array
    """
    
    # fit random forest and print accuracy
    rf = RandomForestClassifier(random_state=0)
    rf.fit(train_data, train_labels)
   
    # list feature importances
    # Get numerical feature importances
    importances = list(rf.feature_importances_)
    feature_importances = pd.DataFrame({'Feature': all_columns,'Coefficient': importances})
    feature_importances = feature_importances.sort_values('Coefficient',ascending=False)
    
    return feature_importances

In [200]:
feature_importances = get_random_forest_n_top_features()
feature_importances

,Feature,Coefficient
28,age_in_years,0.089035
26,monthly_poverty_index,0.083222
19,count_minutes_moderate_sedentary_activity,0.062552
11,alcoholic_drinks_past_12mo,0.050725
1,feeling_down_depressed_hopeless,0.045161
12,how_healthy_is_your_diet,0.038054
9,difficult_doing_daytoday_tasks,0.037632
27,count_hours_worked_last_week,0.037222
13,count_lost_10plus_pounds,0.035683
29,education_level,0.035278


#### Test model performance when using top N features

In [201]:
### Without SMOTE

model_perf_nfeatures = pd.DataFrame(columns = ['model','precision','recall','f1-score','support'])

for number_of_features in list(range(1,len(all_columns))):
    
    # features to use
    # use top N features
    features = list(feature_importances.Feature[:number_of_features])
    
    x_train, x_test, y_train, y_test = get_model_data(
        original_df = cdc_survey_pmom,
        columns = features,
        test_size_prop = 0.2
    )
    
    mod_data = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
    mod_data['n_features'] = number_of_features
    model_perf_nfeatures = model_perf_nfeatures.append(mod_data)    
    
### With SMOTE    
    
model_perf_nfeatures_smote = pd.DataFrame(columns = ['model','precision','recall','f1-score','support'])

for number_of_features in list(range(1,len(all_columns))):
    
    # features to use
    # use top N features
    features = list(feature_importances.Feature[:number_of_features])
    
    x_train, x_test, y_train, y_test = get_model_data(
        original_df = cdc_survey_pmom,
        columns = features,
        test_size_prop = 0.2
    )
    
    mod_data = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
    mod_data['n_features'] = number_of_features
    model_perf_nfeatures_smote = model_perf_nfeatures_smote.append(mod_data)        

In [202]:
alt.Chart(model_perf_nfeatures).mark_line().encode(
    x='n_features:Q',
    y='f1-score:Q',
    color='model:N',
)

alt.Chart(...)

In [113]:
alt.Chart(model_perf_nfeatures_smote).mark_line().encode(
    x='n_features:Q',
    y='f1-score:Q',
    color='model:N',
)

alt.Chart(...)

#### Test model performance when using N features randomly selected

In [208]:
model_perf_nfeatures_rand = pd.DataFrame(columns = ['model','precision','recall','f1-score','support'])

# randomly sample N features
for number_of_features in list(range(1,len(all_columns)+1)):

    # repeat each N 10 times
    for repeat in range(10):
        columns_to_use = random.sample(all_columns, number_of_features)

        x_train, x_test, y_train, y_test = get_model_data(
            original_df = cdc_survey_pmom,
            columns = columns_to_use,
            test_size_prop = 0.2
        )

        mod_data = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
        mod_data['n_features'] = number_of_features
        model_perf_nfeatures_rand = model_perf_nfeatures_rand.append(mod_data)      

In [210]:
model_perf_nfeatures_rand

,model,precision,recall,f1-score,support,n_features
0,Knn,0.581315,0.598117,0.588103,683,1.0
0,Gaussian Naive Bayes,0.633880,0.577534,0.594097,683,1.0
0,Logistic Regression,0.450952,0.500000,0.474211,683,1.0
0,Bernoulli Naive Bayes,0.450952,0.500000,0.474211,683,1.0
0,Random Forest,0.450952,0.500000,0.474211,683,1.0
...,...,...,...,...,...,...
0,Bernoulli Naive Bayes,0.620125,0.688796,0.640166,683,33.0
0,Gaussian Naive Bayes,0.613735,0.664627,0.630484,683,33.0
0,Logistic Regression,0.588745,0.515895,0.510511,683,33.0
0,Knn,0.552096,0.512648,0.506990,683,33.0


In [215]:
# model_perf_nfeatures_rand[(model_perf_nfeatures_rand['model'] == "Knn") & (model_perf_nfeatures_rand['n_features'] == 1)]

In [218]:
alt.Chart(model_perf_nfeatures_rand).mark_point().encode(
    x='n_features:Q',
    y='f1-score:Q',
    color='model:N',
).facet(
    facet='model:N',
    columns=2
)

alt.FacetChart(...)